# Imports

In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
from NuRadioReco.modules.io.rno_g import rnogDataReader
from astropy.time import Time
from NuRadioReco.utilities.fft import time2freq
from NuRadioReco.utilities import units
import pandas as pd
from NuRadioReco.detector import detector
from datetime import datetime

from NuRadioReco.modules.base.module import register_run
from NuRadioReco.utilities import units

from NuRadioReco.framework.trigger import SimplePhasedTrigger
from NuRadioReco.modules.analogToDigitalConverter import analogToDigitalConverter
import logging
import scipy
import numpy as np
from scipy import constants

import logging
logging.basicConfig(level=0)
from NuRadioMC.SignalProp import propagation
from NuRadioMC.utilities import medium
import scipy.optimize
import NuRadioReco.utilities.geometryUtilities
from scipy.constants import speed_of_light
speed_of_light /= units.s
from radiotools import helper as hp

from NuRadioReco.framework.base_trace import BaseTrace
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange

from NuRadioReco.utilities.geometryUtilities import get_fresnel_angle


from scipy.signal import find_peaks
from scipy.optimize import curve_fit

from iminuit import Minuit
from iminuit import minimize

from gpxplotter import read_gpx_file, create_folium_map, add_segment_to_map
import datetime
from dateutil.tz import tzutc
import copy
import numpy as np
import utm
import matplotlib.pyplot as plt
import os
the_map = create_folium_map(tiles='opentopomap')

cspeed = constants.c * units.m / units.s

import matplotlib.dates as mdates
from datetime import datetime
from matplotlib.colors import Normalize

# Balloon

In [2]:
# Plots Balloon tracks with station positions in Lon/Lat
def BalloonTracks_LonLat(filename):
    gpx_file_name = filename
    i = 0
    for track in read_gpx_file(gpx_file_name):
        continue
    segment = track['segments'][0]
    
    plt.plot(segment["lon"], segment["lat"],label='weatherballoon track')
    plt.plot(lon_stat, lat_stat, 'r.',label='running stations')
    plt.ylabel("latitude [deg]")
    plt.xlabel("longitude [deg]")
    plt.legend()
    #plt.savefig('test.pdf')
    #plt.figure()


# Plots Balloon tracks with station positions in UTM-koordinates 
def BalloonTracks_UTM(filename):
    gpx_file_name = filename
    i = 0
    for track in read_gpx_file(gpx_file_name):
        continue
    segment = track['segments'][0]
    
    data=utm.from_latlon(segment["lat"], segment["lon"])
    x_data=data[0]
    y_data=data[1]
    plt.figure()
    
    plt.plot(station[:,3],station[:,4], 'r.',label='running stations')

    plt.plot(x_data,y_data, label='weatherballoon track')

    plt.xlabel("meters East (Zone 24 X)")
    plt.ylabel("meters North (Zone 24 X)")
    plt.legend()
    
# Calculates angle between balloon and stations
def angleBalloonStation(x_stat, y_stat, z_stat, x_ba, y_ba, z_ba):
    length=np.sqrt((x_stat-x_ba)**2+(y_stat-y_ba)**2)
    height=np.sqrt((z_stat-z_ba)**2)
    #Noch zu bachten: Bezug zu Höhe überm Meeresspiegel und Tiefe der Antenne im Eis 
    return np.pi/2-np.arctan(height/length)

# Calculates 3d distance between balloon and stations 
def dist_StationBalloon(x_stat, y_stat, z_stat,x_ba, y_ba, z_ba):
    return np.sqrt((x_stat-x_ba)**2+(y_stat-y_ba)**2+(z_stat-z_ba)**2)

# Calculates 2d distance between balloon and stations
def dist_StationBalloon_2d(x_stat, y_stat,x_ba, y_ba):
    return np.sqrt((x_stat-x_ba)**2+(y_stat-y_ba)**2)

# Phasing

In [9]:
import logging
from NuRadioReco.utilities import units

# In analogy to https://github.com/nu-radio/NuRadioMC/blob/develop/NuRadioReco/modules/phasedarray/triggerSimulator.py#L65
class Phasing:

    def __init__(self, log_level=logging.WARNING):
        self.__t = 0
        self.__pre_trigger_time = None
        self.__debug = None
        logger.setLevel(log_level)
        self.begin()

    def begin(self, debug=False, pre_trigger_time=100 * units.ns):
        self.__pre_trigger_time = pre_trigger_time
        self.__debug = debug

    def get_antenna_positions(self, station, det, used_channels, component):
        ant_pos = {}
        for channel in station.iter_channels(use_channels=used_channels):
            ant_pos[channel.get_id()] = det.get_relative_position(station.get_id(), channel.get_id())[component]
        return ant_pos  
    
    # modified part

    def calculate_time_delays(self, station, det,
                              used_channels,
                              sampling_frequency,
                              phasing_angles,
                              ref_index=1.75):

    
        time_step = 1. / sampling_frequency
    
        ant_z = self.get_antenna_positions(self, station, det, used_channels, 2)
        ref_z = np.max(np.fromiter(ant_z.values(), dtype=float))
        min_z_pos = np.min(np.fromiter(ant_z.values(), dtype=float))
        
        phys_max_delay_wo_cables = (ref_z-min_z_pos)/ cspeed * ref_index
        

        # Need to add in delay for trigger delay
        cable_delays = {}
        for channel in station.iter_channels(use_channels=used_channels):
            cable_delays[channel.get_id()] = det.get_cable_delay(station.get_id(), channel.get_id())
            
        
        angle_rolls_wodict=[]
        
        for angle in phasing_angles:
            phys_max_delay=[]
            delays = []
            phys_pos_delays=[]
            num_angles=[]
            pos_angle= False
            for key in ant_z:
                delays += [-(ant_z[key] - ref_z) / cspeed * ref_index * np.cos(angle) - cable_delays[key]]
                
                # calculation of physically possible max delay
                
                phys_max_delay += [-(ant_z[key] - ref_z) / cspeed * ref_index - cable_delays[key]]
    
           
            if (delays[0]<=phys_max_delay[0]and delays[1]<=phys_max_delay[1]and delays[2]<=phys_max_delay[2]and delays[3]<=phys_max_delay[3]):
                phys_pos_delays += delays
                pos_angle= True 
            
            phys_pos_delays -= np.max(phys_pos_delays)

            roll = np.array(np.round(np.array(phys_pos_delays) / time_step)).astype(int)
            for i in range (len(used_channels)):
                angle_rolls_wodict.append(roll[i])
            if(pos_angle==True):
                num_angles.append(angle)
                angle_rolls_wodict.append(angle)
            
            angle_array=np.array(angle_rolls_wodict)
            
        
        angle_narray=angle_array.reshape([len(phasing_angles),len(used_channels)+1])

        return angle_narray

In [12]:
def get_phasing_peaks_power_and_angle_complete_PA(reader,bandpass_low, bandpass_high, sampling_rate, delays, phasing_angles, start_index, stop_index):
    
    """
    calculates power and corresponding PA angle for peaks in coherent sum of filtered and umpsampled traces of all channels
    
    Arguments:
    
    - reader:
        reader for used data
    
    - bandpass_low:
        lower limit of bandpass filter
    
    - bandpass_high:
        higher limit of bandpass filter
        
    - sampling_rate:
        sampling rate of the traces
        
    - delays_all_channels:
        calculated delays as numpy array of shape [len(phasing_angles),5]
        
    - phasing_angles:
        list of angles for which delays are given
    
    - start_index:
        first event id for which the function is called
        
    - stop_index:
        last event id for which the function is NOT called
        
    Returns:
    - station time of events
    - total power for all delays
        
    
    """

    max_power=[]
    event_times=[]
    ges_power=[]
    power_of_peaks=[]
    angles_of_peaks=[]
    
    
    
    for i, event in enumerate(reader.get_events()):
        station = event.get_station(event.get_station_ids()[0])
        channel0 = station.get_channel(0)
        channel0.resample(sampling_rate)
        trace0=channel0.get_filtered_trace([bandpass_low, bandpass_high])
        channel1= station.get_channel(1)
        channel1.resample(sampling_rate)
        trace1=channel1.get_filtered_trace([bandpass_low, bandpass_high])
        channel2 = station.get_channel(2)
        channel2.resample(sampling_rate)
        trace2=channel2.get_filtered_trace([bandpass_low, bandpass_high])
        channel3 = station.get_channel(3)
        channel3.resample(sampling_rate)
        trace3=channel3.get_filtered_trace([bandpass_low, bandpass_high])
    
    
        if i>start_index:
            
    
            
            
            baseTrace_sum= BaseTrace()
            power=[]
            time=station.get_station_time()
            event_times.append(time)
        
        
            for j in range (delays.shape[0]):

                shift0=np.roll(trace0,round(delays[j,0]))
                shift1=np.roll(trace1,round(delays[j,1]))
                shift2=np.roll(trace2,round(delays[j,2]))
                shift3=np.roll(trace3,round(delays[j,3]))
                shifted_sum=(shift0+shift1+shift2+shift3)[round((sampling_rate)/(3.2*units.GHz))*250:-round((sampling_rate)/(3.2*units.GHz))*250]
                baseTrace_sum.set_trace(shifted_sum, sampling_rate)
                bin_403=np.argmin(abs(baseTrace_sum.get_frequencies()-403*units.MHz)) 
                power.append((abs(baseTrace_sum.get_frequency_spectrum()[bin_403]))**2)
            
            
        
            peaks, _ = find_peaks(power, height=np.max(power)/1.2)
            ges_power.append(power)
            #power_of_peaks.append(power[peaks])
            #angles_of_peaks.append(phasing_angles[peaks])
        
        if i>stop_index:
            break
            
    
    return event_times,ges_power

# Expected zenith angle at PA

In [5]:
def get_zenith_phased_array(d, h, antenna_depth=-100,ice_model='greenland_simple'):
    """
    calculates zenith angle at phased arry for given balloon position
    
    Arguments:
    - d:
        balloon radius from station center
    - h:
        elevation of balloon over surface
    - antenna_depth: 
        depth of antenna in ice
    - ice_model:
        ice model used for raytracing
        
    Returns:
    - zenith at PA
    """
    
    ice = medium.get_ice_model(ice_model)
    raytracing = propagation.get_propagation_module('analytic')
    rt = raytracing(ice)
    
    def chisq(x):
        
        zenith_balloon_x_geometric=np.arctan((d-x)/h)
        zenith_balloon_x_in_ice=get_fresnel_angle(zenith_balloon_x_geometric, ice.get_index_of_refraction([0,0,0]))
        rt.set_start_and_end_point([x,0,0],[0,0,antenna_depth])
        rt.find_solutions()
        if not rt.has_solution():
            return(2*np.pi)
        

        start_angle=np.pi - hp.cartesian_to_spherical(*rt.get_launch_vector(0))[0]
        return (zenith_balloon_x_in_ice-start_angle)**2
    
    res = scipy.optimize.fmin(chisq, x0=50, disp=False)
    x = res[0]
    rt.set_start_and_end_point([x,0,0],[0,0,antenna_depth])
    rt.find_solutions()
    pa_angle=hp.cartesian_to_spherical(*rt.get_receive_vector(0))[0]
    
    return pa_angle

# Calculation of surface interaction point PA angle as starting point

In [6]:
def get_surface_interaction_point_and_surface_angle_in_air(pa_angle, antenna_depth=-98,ice_model='greenland_simple'):
    """
    calculates surface interaction point and the corresponding angle at surface in air
    
    Arguments:
    -pa_angle:
        angle at phased array
    - antenna_depth: 
        depth of antenna in ice
    - ice_model:
        ice model used for raytracing
        
    Returns:
    - surface interaction point as distance to phased array
    - angle at surface in air
    """
    
    ice = medium.get_ice_model(ice_model)
    raytracing = propagation.get_propagation_module('analytic')
    rt = raytracing(ice)
    
    def chisq(x):
        
        rt.set_start_and_end_point([0,0,antenna_depth],[x,0,0])
        rt.find_solutions()
        if not rt.has_solution():
            return(2*np.pi)
        

        start_angle=hp.cartesian_to_spherical(*rt.get_launch_vector(0))[0]
        return (pa_angle-start_angle)**2
    
    res = scipy.optimize.fmin(chisq, x0=50, disp=False)
    x = res[0]
    rt.set_start_and_end_point([0,0,antenna_depth],[x,0,0])
    rt.find_solutions()
    surface_angle=get_fresnel_angle(np.pi-hp.cartesian_to_spherical(*rt.get_receive_vector(0))[0],1,ice.get_index_of_refraction([0,0,0]))
    
    return x, surface_angle

# Fitting 

## with auxiliary variables

In [7]:
class BalloonRecoFit:
    def __init__(self):
        self.station_pos = None
        self.xpa = None
        self.ypa = None
        self.zpa = None
        self.theta = None
        self.zs = None
        self.rho = None
        self.err = None
    
    #def SetParams(self, station_pos):
    
    def SetPos(self, station_pos):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        #if nstat < 3:
        #    print('ERROR, need at least three sensors ...')
        #    goodfit = False

        #else:
        #    print('Number of measured positions:', nstat)
        #    print('Measured positions:')
        #    print(self.station_pos)
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    def SetParams(self, station_pos, theta0, theta1, theta2, zs, rho0, rho1, rho2, err0, err1, err2, err3, err4, err5):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    
        self.theta=[theta0,theta1,theta2]
        self.zs=zs
        self.rho=[rho0, rho1, rho2]
        self.err=[err0, err1, err2, err3, err4, err5]
        
        
    
    def function_to_minimize(self,xb, yb, zb, theta0, theta1,theta2, zs, phi0, phi1, phi2, rho0, rho1, rho2):
        deltad = 1
        res = 0
        #res += np.sqrt(
        #    (np.tan(theta0) - (np.sqrt((xb - xs0) ** 2 + (yb - ys0)**2) / (zb - zs))) ** 2 + 
        #    (np.tan(theta1) - (np.sqrt((xb - xs1) ** 2 + (yb - ys1)**2) / (zb - zs))) ** 2 +
        #    (np.tan(theta1) - (np.sqrt((xb - xs2) ** 2 + (yb - ys2)**2) / (zb - zs))) ** 2 +
        #    (xs0 / ys0 - (rho0 * np.cos(phi0) - self.xpa[0]) / (rho0 * np.sin(phi0) - self.ypa[0])) ** 2 +
        #    (xs1 / ys1 - (rho1 * np.cos(phi1) - self.xpa[1]) / (rho1 * np.sin(phi1) - self.ypa[1])) ** 2 +
        #    (xs2 / ys2 - (rho2 * np.cos(phi2) - self.xpa[2]) / (rho1 * np.sin(phi2) - self.ypa[2])) ** 2
        #    ) / (deltad * deltad)
        
        res += np.sqrt(
            (np.tan(theta0) - (np.sqrt((xb - (rho0*np.cos(phi0)+self.xpa[0])) ** 2 + (yb - (rho0*np.sin(phi0)+self.ypa[0]))**2) / (zb - zs))) ** 2 + 
            (np.tan(theta1) - (np.sqrt((xb - (rho1*np.cos(phi1)+self.xpa[1])) ** 2 + (yb - (rho1*np.sin(phi1)+self.ypa[1]))**2) / (zb - zs))) ** 2 +   
            (np.tan(theta2) - (np.sqrt((xb - (rho2*np.cos(phi2)+self.xpa[2])) ** 2 + (yb - (rho2*np.sin(phi2)+self.ypa[2]))**2) / (zb - zs))) ** 2 + 
            (np.tan(phi0)-(yb-self.ypa[0])/(xb-self.xpa[0]))**2 +
            (np.tan(phi1)-(yb-self.ypa[1])/(xb-self.xpa[1]))**2 +
            (np.tan(phi2)-(yb-self.ypa[2])/(xb-self.xpa[2]))**2 
            )/(deltad * deltad)
        
        return res
    
    def function_to_minimize_with_errors(self,xb, yb, zb, phi0, phi1, phi2):
        deltad = 1
        res = 0
        #res += np.sqrt(
        #    (np.tan(theta0) - (np.sqrt((xb - xs0) ** 2 + (yb - ys0)**2) / (zb - zs))) ** 2 + 
        #    (np.tan(theta1) - (np.sqrt((xb - xs1) ** 2 + (yb - ys1)**2) / (zb - zs))) ** 2 +
        #    (np.tan(theta1) - (np.sqrt((xb - xs2) ** 2 + (yb - ys2)**2) / (zb - zs))) ** 2 +
        #    (xs0 / ys0 - (rho0 * np.cos(phi0) - self.xpa[0]) / (rho0 * np.sin(phi0) - self.ypa[0])) ** 2 +
        #    (xs1 / ys1 - (rho1 * np.cos(phi1) - self.xpa[1]) / (rho1 * np.sin(phi1) - self.ypa[1])) ** 2 +
        #    (xs2 / ys2 - (rho2 * np.cos(phi2) - self.xpa[2]) / (rho1 * np.sin(phi2) - self.ypa[2])) ** 2
        #    ) / (deltad * deltad)
        
    
        
        res += np.sqrt(
            ((np.tan(self.theta[0]) - (np.sqrt((xb - (self.rho[0]*np.cos(phi0)+self.xpa[0])) ** 2 + (yb - (self.rho[0]*np.sin(phi0)+self.ypa[0]))**2) / (zb - self.zs)))/self.err[0]) ** 2 + 
            ((np.tan(self.theta[1]) - (np.sqrt((xb - (self.rho[1]*np.cos(phi1)+self.xpa[1])) ** 2 + (yb - (self.rho[1]*np.sin(phi1)+self.ypa[1]))**2) / (zb - self.zs)))/self.err[1]) ** 2 +   
            ((np.tan(self.theta[2]) - (np.sqrt((xb - (self.rho[2]*np.cos(phi2)+self.xpa[2])) ** 2 + (yb - (self.rho[2]*np.sin(phi2)+self.ypa[2]))**2) / (zb - self.zs)))/self.err[2]) ** 2 + 
            ((np.tan(phi0)-(yb-self.ypa[0])/(xb-self.xpa[0]))/self.err[3])**2 +
            ((np.tan(phi1)-(yb-self.ypa[1])/(xb-self.xpa[1]))/self.err[4])**2 +
            ((np.tan(phi2)-(yb-self.ypa[2])/(xb-self.xpa[2]))/self.err[5])**2 
            )
        
        return res
    
    def test_function_to_minimize_2stat(self,xb, yb, zb, theta0, theta1, xs0, xs1, ys0, ys1, zs, phi0, phi1, rho0, rho1):
        deltad = 1
        res = 0
        res += np.sqrt(
            (theta0 - (np.sqrt((xb - xs0) ** 2 + (yb - ys0)) / (zb - zs))) ** 2 +
            (theta1 - (np.sqrt((xb - xs1) ** 2 + (yb - ys1)) / (zb - zs))) ** 2 +
            (xs0 / ys0 - (rho0 * np.cos(phi0) - self.xpa[0]) / (rho0 * np.sin(phi0) - self.ypa[0])) ** 2 +
            (xs1 / ys1 - (rho1 * np.cos(phi1) - self.xpa[1]) / (rho1 * np.sin(phi1) - self.ypa[1])) ** 2
            )/(deltad * deltad)
        return res
    
    def actual_Fit_2stat(self,fit_values):

        
        m = Minuit(self.test_function_to_minimize_2stat,*fit_values) 
        
        m.fixed[3] = True
        m.fixed[4] = True
        m.fixed[9] = True
        m.fixed[12] = True
        m.fixed[13] = True
        
        m.migrad()  
        
        #m.hesse() 

        #print(m.params)
        
        finvalues = m.values
        #finerrors = m.errors
        
        print('finals values from fit:')
        print(finvalues)
        
        return finvalues
    
    def actual_Fit(self,fit_values):

        
        m = Minuit(self.function_to_minimize,*fit_values) 
        
        m.fixed[3] = True
        m.fixed[4] = True
        m.fixed[5] = True
        
     
        m.fixed[6] = True
        
        m.fixed[10] = True
        m.fixed[11] = True
        m.fixed[12] = True
        
        
        m.migrad()  
        
        m.hesse() 

        print(m.params)
        
        finvalues = m.values
        #finerrors = m.errors
        
        #print('finals values from fit:')
        #print(finvalues)
        
        return finvalues
    
    
    def actual_Fit_with_errors(self,fit_values):

        
        m = Minuit(self.function_to_minimize_with_errors,*fit_values) 
        
        m.migrad()  
        
        m.hesse() 

        print(m.params)
        
        finvalues = m.values
        #finerrors = m.errors
        
        #print('finals values from fit:')
        #print(finvalues)
        
        return finvalues

## simple fit

In [8]:
class BalloonRecoFit:
    def __init__(self):
        self.station_pos = None
        self.xpa = None
        self.ypa = None
        self.zpa = None
        self.theta = None
        self.zs = None
        self.rho = None
        self.err = None
        self.err_xpa = None
        self.err_ypa = None
        self.err_theta = None
        self.err_rho = None
        self.err_zs = None
    
    #def SetParams(self, station_pos):
    
    def SetPos(self, station_pos):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        #if nstat < 3:
        #    print('ERROR, need at least three sensors ...')
        #    goodfit = False

        #else:
        #    print('Number of measured positions:', nstat)
        #    print('Measured positions:')
        
        #    print(self.station_pos)
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    def SetParams(self, station_pos, theta0, theta1, theta2, zs, rho0, rho1, rho2, err0, err1, err2):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    
        self.theta=[theta0,theta1,theta2]
        self.zs=zs
        self.rho=[rho0, rho1, rho2]
        self.err=[err0, err1, err2]

    def SetParams_4stat(self, station_pos, theta0, theta1, theta2, theta3, zs, rho0, rho1, rho2, rho3, err0, err1, err2, err3):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    
        self.theta=[theta0,theta1,theta2,theta3]
        self.zs=zs
        self.rho=[rho0, rho1, rho2,rho3]
        self.err=[err0, err1, err2,err3]
        
        
    def SetParams_5stat(self, station_pos, theta0, theta1, theta2, theta3,theta4, zs, rho0, rho1, rho2, rho3,rho4, err0, err1, err2, err3, err4):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
    
    
        self.theta=[theta0,theta1,theta2,theta3, theta4]
        self.zs=zs
        self.rho=[rho0, rho1, rho2,rho3,rho4]
        self.err=[err0, err1, err2,err3,err4]
        
    def SetParams_no_err(self, station_pos, theta, zs, rho):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
        
        self.theta = []  
        self.rho = [] 
        self.zs = zs 
        
        
        
        for i in range (len(self.station_pos.items())):  
            self.theta.append(theta[i])
            #self.zs.append(zs[i])
            self.rho.append(rho[i])
            
        
    def SetParams_w_err(self, station_pos, theta, zs, rho, err_xpa, err_ypa,err_theta,err_zs, err_rho):
        self.station_pos = {}
        for station in station_pos.keys():
            if len(station_pos[station]) > 0:
                self.station_pos[station] = copy.copy(station_pos[station])
        nstat = len(self.station_pos.keys())
        
        
        print('Number of measured positions:', nstat)
        print('Measured positions:')
        print(self.station_pos)
        
        self.xpa = []  
        self.ypa = []  
        self.zpa = []  
        
        for station_id, pos in self.station_pos.items():
            xpa, ypa, zpa = pos 
            self.xpa.append(xpa)
            self.ypa.append(ypa)
            self.zpa.append(zpa)
        
        self.err_xpa = []  
        self.err_ypa = []  
        self.err_zs = []
        self.err_theta = []  
        self.err_rho = [] 
        self.theta = []  
        self.rho = [] 
        self.zs = zs 
        
        
        
        for i in range (len(self.station_pos.items())):  
            self.theta.append(theta[i])
            self.rho.append(rho[i])
            self.err_xpa.append(err_xpa[i])
            self.err_ypa.append(err_ypa[i])
            self.err_theta.append(err_theta[i])
            self.err_zs.append(err_zs[i])
            self.err_rho.append(err_rho[i])
        
        
    
    
    def function_to_minimize_no_ad_variables(self,xb, yb, zb):
        
        res = 0
        for station in range (len(self.theta)):
            res =  (res + ((np.tan(self.theta[station])*(zb - self.zs)+self.rho[station]
                    - np.sqrt((xb - self.xpa[station]) ** 2 + (yb - self.ypa[station])**2))/self.err[station]) ** 2)
        
        
        return res
    
    
    def actual_Fit_no_ad_variable(self,fit_values):

        
        m = Minuit(self.function_to_minimize_no_ad_variables,*fit_values) 
        
        m.migrad()  
        
        m.hesse() 
        print('Hesse:')
        print(m.hesse())
        print('Covariance correlation:')
        print(m.covariance.correlation())
        finvalues = m.values
        
        return finvalues
    
    
    def function_to_minimize_error_func(self,xb, yb, zb):
        
        res = 0
        for station in range (len(self.theta)):
            res =  (res + ((np.tan(self.theta[station])*(zb - self.zs)+self.rho[station]
                    - np.sqrt((xb - self.xpa[station]) ** 2 + (yb - self.ypa[station])**2))/
                          
                    np.sqrt(((zb - self.zs)*(np.tan(self.theta[station])**2 + 1)*self.err_theta[station])**2+
                    self.err_rho[station]**2+
                    ((-(-xb + self.xpa[station])/np.sqrt((xb - self.xpa[station])**2 + (yb - self.ypa[station])**2))*self.err_xpa[station])**2+
                    ((-(-yb + self.ypa[station])/np.sqrt((xb - self.xpa[station])**2 + (yb - self.ypa[station])**2))*self.err_ypa[station])**2+
                    (-np.tan(self.theta[station])*self.err_zs[station])**2)
    
                          ) ** 2)
        
        
        return res
    
    
    def actual_Fit_error_func(self,fit_values):

        
        m = Minuit(self.function_to_minimize_error_func,*fit_values) 
        
        m.migrad()  
        
        m.hesse() 
        print('Hesse:')
        print(m.hesse())
        print('Covariance correlation:')
        print(m.covariance.correlation())
        finvalues = m.values
        
        return finvalues

